### Definition Generation Evaluation

In [13]:
import json
# model = "vicuna"
model = "llama"

# Load the JSON data
with open(f'./dataset/{model}_definition_combined.json', 'r') as json_file:
    data = json.load(json_file)

#### Dataset creation

In [14]:
# Initialize lists to store predictions and references
predictions = []
references = []
references_bluert = []

# Iterate through each item in the JSON data
for item in data:
  celex_id = item["celex_id"]
  if item["existing_record"] == ["NEW TERM"]:
    original_text = item['original_definition']
    term = item['term']
    new_term = f"'{term}'"
    original_text = original_text.replace(term, new_term)
    generated_text = item['generated_definition']

    if ((generated_text != "") and (generated_text != "NO JSON AS AN OUTPUT OBTAINED")):
      # Add the generated text to predictions list
      predictions.append(generated_text)

      # Add the original text to references list
      references_item = [original_text]  # References for this item
      references.append(references_item)

      references_bluert.append(original_text)

In [15]:
!pip install evaluate

#### BLEU Score
- Reason for choosing
  
  - Length of the generated definitions matters
  - Precise definition is required


In [5]:
import evaluate
bleu = evaluate.load("bleu")

/Users/ashishchouhan/Desktop/PhD/6a.Production_Code/4. final_paper_code/LexDrafter/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# Compute the BLEU score
bleu4_results = bleu.compute(predictions=predictions, references=references, max_order=4)
print(f"BLEU-4: {bleu4_results['bleu']}")

BLEU-4: 0.054211222286845544


In [7]:
# Compute the BLEU score
bleu3_results = bleu.compute(predictions=predictions, references=references, max_order=3)
print(f"BLEU-3: {bleu3_results['bleu']}")

BLEU-3: 0.08512705418500642


In [8]:
# Compute the BLEU score
blue2_results = bleu.compute(predictions=predictions, references=references, max_order=2)
print(f"BLEU-2: {blue2_results['bleu']}")

BLEU-2: 0.1470442594318926


In [9]:
# Compute the BLEU score
blue1_results = bleu.compute(predictions=predictions, references=references, max_order=1)
print(f"BLEU-1: {blue1_results['bleu']}")

BLEU-1: 0.27602616866006596


#### BERTScore

In [16]:
!pip install bert_score

In [17]:
import evaluate
bertscore = evaluate.load("bertscore")

ModuleNotFoundError: No module named 'evaluate'

In [ ]:
# bertscore_results = bertscore.compute(predictions=predictions, references=references, model_type="distilbert-base-uncased")
bertscore_results = bertscore.compute(predictions=predictions, references=references, model_type="nlpaueb/legal-bert-base-uncased")

NameError: name 'bertscore' is not defined

In [ ]:
# Extract the F1 scores
f1_scores = bertscore_results['f1']

# Calculate the mean of F1 scores
mean_f1 = sum(f1_scores) / len(f1_scores)

print(f"BERTScore_F1: {bertscore_results['f1']}")
print("--------")
print(f"Mean BERTScore F1: {mean_f1}")

BERTScore_F1: [0.771040678024292, 0.802032470703125, 0.7775539755821228, 0.7847458720207214, 0.7986399531364441, 0.8132898807525635, 0.7352892756462097, 0.8096631765365601, 0.7902698516845703, 0.8768617510795593, 0.8628918528556824, 0.8763148784637451, 0.7551087737083435, 0.8277158737182617, 0.8119568228721619, 0.8192282915115356, 0.8277431130409241, 0.7887316942214966, 0.8555272817611694, 0.8275467753410339, 0.80133455991745, 0.7832514047622681, 0.896211564540863, 0.8665550947189331, 0.7223899960517883, 0.7245991826057434, 0.8335634469985962, 0.8275464177131653, 0.852604866027832, 0.8663516044616699, 0.7545173764228821, 0.7698800563812256, 0.8558621406555176, 0.8944920897483826, 0.8309657573699951, 0.8162767887115479, 0.8207405805587769, 0.8322715759277344, 0.857930600643158, 0.8114943504333496, 0.871566653251648, 0.8979578614234924, 0.8206650614738464, 0.79401034116745, 0.8262958526611328, 0.8130106329917908, 0.788678765296936, 0.8262680172920227, 0.7483661770820618, 0.71179884672164

In [10]:
# Extract the precision scores
precision_scores = bertscore_results['precision']

# Calculate the mean of precision scores
mean_precision = sum(precision_scores) / len(precision_scores)

print(f"BERTScore_precision: {bertscore_results['precision']}")
print("--------")
print(f"Mean BERTScore precision: {mean_precision}")

NameError: name 'bertscore_results' is not defined

In [11]:
# Extract the recall scores
recall_scores = bertscore_results['recall']

# Calculate the mean of recall scores
mean_recall = sum(recall_scores) / len(recall_scores)

print(f"BERTScore_recall: {bertscore_results['recall']}")
print("--------")
print(f"Mean BERTScore recall: {mean_recall}")

NameError: name 'bertscore_results' is not defined

#### BLEURT

In [16]:
!pip install git+https://github.com/google-research/bleurt.git

  Cloning https://github.com/google-research/bleurt.git to /private/var/folders/rq/lmq0w5vs1_nfq3rrr7wpx3th0000gn/T/pip-req-build-7b_ariu9
  Running command git clone --filter=blob:none --quiet https://github.com/google-research/bleurt.git /private/var/folders/rq/lmq0w5vs1_nfq3rrr7wpx3th0000gn/T/pip-req-build-7b_ariu9
  Resolved https://github.com/google-research/bleurt.git to commit cebe7e6f996b40910cfaa520a63db47807e3bf5c
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 44.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.0/227.0 MB 29.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.1/352.1 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 27.0 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.8/389.8 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [17]:
import evaluate
bleurt = evaluate.load("bleurt", 'BLEURT-20')

INFO:tensorflow:Reading checkpoint /Users/ashishchouhan/.cache/huggingface/metrics/bleurt/BLEURT-20/downloads/extracted/5e239c156272523c25a71510af4ce974a3b2b17344901cd941228a3ecd2d36d0/BLEURT-20.
INFO:tensorflow:Config file found, reading.
INFO:tensorflow:Will load checkpoint BLEURT-20
INFO:tensorflow:Loads full paths and checks that files exists.
INFO:tensorflow:... name:BLEURT-20
INFO:tensorflow:... bert_config_file:bert_config.json
INFO:tensorflow:... max_seq_length:512
INFO:tensorflow:... vocab_file:None
INFO:tensorflow:... do_lower_case:None
INFO:tensorflow:... sp_model:sent_piece
INFO:tensorflow:... dynamic_seq_length:True
INFO:tensorflow:Creating BLEURT scorer.
INFO:tensorflow:Creating SentencePiece tokenizer.
INFO:tensorflow:Creating SentencePiece tokenizer.
INFO:tensorflow:Will load model: /Users/ashishchouhan/.cache/huggingface/metrics/bleurt/BLEURT-20/downloads/extracted/5e239c156272523c25a71510af4ce974a3b2b17344901cd941228a3ecd2d36d0/BLEURT-20/sent_piece.model.
INFO:tensorf

INFO:tensorflow:BLEURT initialized.


In [18]:
# Compute the BLEURT score
bleurt_results = bleurt.compute(predictions=predictions, references=references_bluert)

In [19]:
# Extract the scores
scores = bleurt_results['scores']

# Calculate the mean of F1 scores
mean_bleurt = sum(scores) / len(scores)

In [20]:
print(f"BLEURT: {bleurt_results['scores']}")
print("--------")
print(f"Mean BLEURT: {mean_bleurt}")

BLEURT: [0.3606106638908386, 0.3854888677597046, 0.401622474193573, 0.6615625619888306, 0.4073513150215149, 0.45376765727996826, 0.38676947355270386, 0.45257604122161865, 0.4630216956138611, 0.691747784614563, 0.5683438181877136, 0.5162121057510376, 0.44876378774642944, 0.6059715151786804, 0.5735872387886047, 0.529415488243103, 0.4726722240447998, 0.4412979483604431, 0.4777666926383972, 0.481040894985199, 0.4596937298774719, 0.3912540674209595, 0.714878261089325, 0.6160227060317993, 0.3207806944847107, 0.4370354413986206, 0.5518521666526794, 0.46938949823379517, 0.6072700619697571, 0.6152352094650269, 0.3979171812534332, 0.45518457889556885, 0.45618778467178345, 0.5681602954864502, 0.5568466782569885, 0.36769235134124756, 0.40374159812927246, 0.4004210829734802, 0.520168662071228, 0.3059939742088318, 0.572227418422699, 0.6877575516700745, 0.6018601059913635, 0.4764571785926819, 0.4242071509361267, 0.567833662033081, 0.45518553256988525, 0.4765969514846802, 0.4106636047363281, 0.2047957